<a href="https://colab.research.google.com/github/YuriArduino/Estudos_Artificial_Intelligence/blob/main/pandas_workflow_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas Workflow com LlamaIndex Workflows

>Este notebook demonstra como criar um sistema de an√°lise de dados usando LlamaIndex, Pandas e Groq para consultas em linguagem natural.


## Sobre a Atualiza√ß√£o do LlamaIndex

O **LlamaIndex** descontinuou o m√≥dulo **QueryPipeline** em favor de uma nova abordagem chamada **Workflows**.
Essa mudan√ßa foi implementada na vers√£o `0.11` do LlamaIndex, com o objetivo de oferecer uma arquitetura mais
flex√≠vel e escal√°vel para a constru√ß√£o de aplica√ß√µes de IA generativa.

### Por que o QueryPipeline foi descontinuado?
O QueryPipeline era uma API declarativa √∫til para orquestrar consultas simples a avan√ßadas sobre os dados.
Por√©m, ele se mostrou limitado para cen√°rios mais din√¢micos. Por isso, os **Workflows** foram introduzidos,
trazendo uma arquitetura orientada a eventos para fluxos de trabalho mais sofisticados.

### O que s√£o Workflows?
Workflows permitem orquestrar etapas personalizadas de forma ass√≠ncrona e condicional, facilitando
integra√ß√µes complexas e manipula√ß√£o de dados em tempo real.

**Documenta√ß√£o oficial:** [docs.llamaindex.ai](https://docs.llamaindex.ai)



#C√≥digo:


##1 Instalar Depend√™ncias:

In [1]:
# Instalar depend√™ncias (-q para instala√ß√£o silenciosa)
!pip install -q jedi>=0.16 llama-index llama-index-llms-openai llama-index-llms-groq llama-index-experimental gradio fpdf

##1.1 Para uso no Colab


In [2]:
import nest_asyncio
nest_asyncio.apply()

##2 Imports

In [3]:
import pandas as pd
import textwrap
import re
from pydantic import BaseModel, Field, field_validator, ConfigDict, ValidationError
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.core.workflow import Workflow, Event, StartEvent as BaseStartEvent, StopEvent, step
from llama_index.core.prompts import PromptTemplate
from google.colab import userdata
from typing import Any, List, Tuple, Optional
import asyncio
import gradio as gr
from fpdf import FPDF
from datetime import datetime

##3 Instru√ß√µes e Prompts

(Importante para validar a sa√≠da, pode ser usado como debug ou  )

In [4]:
# ===== INSTRU√á√ïES PARA CONVERS√ÉO DE CONSULTAS =====
instruction_str = (
    "1. Converta a consulta para c√≥digo Python execut√°vel usando Pandas.\n"
    "2. A linha final do c√≥digo deve ser uma express√£o Python que possa ser chamada com a fun√ß√£o `eval()`.\n"
    "3. O c√≥digo deve representar uma solu√ß√£o para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESS√ÉO FINAL.\n"
    "5. N√£o coloque a express√£o entre aspas.\n"
    "6. Evite atribui√ß√µes (=) na linha final - prefira express√µes que retornem valores.\n"
    "7. Para opera√ß√µes de m√∫ltiplas linhas, termine com uma express√£o que retorne o resultado.\n"
    "8. Exemplos v√°lidos:\n"
    "   - df.groupby('coluna')['valor'].sum()\n"
    "   - df['coluna'].value_counts().head(5)\n"
    "   - df.describe()\n"
    "9. Evite c√≥digos como 'df['coluna'] = valor' - prefira consultas que retornem dados.\n"
)

# ===== PROMPTS =====
pandas_prompt_str = (
    "Voc√™ est√° trabalhando com um dataframe do pandas em Python chamado `df`.\n"
    "{colunas_detalhes}\n\n"
    "Este √© o resultado de `print(df.head())`:\n"
    "{df_str}\n\n"
    "Siga estas instru√ß√µes:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Express√£o:"
)

RESPONSE_SYNTHESIS_PROMPT_STR = (
   "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
   "Responda de forma natural, sem introdu√ß√µes como 'A resposta √©:' ou algo semelhante.\n"
   "Consulta: {query_str}\n\n"
   "Instru√ß√µes do Pandas (opcional):\n{pandas_instructions}\n\n"
   "Sa√≠da do Pandas: {pandas_output}\n\n"
   "Resposta: "
   "Ao final, exibir o c√≥digo usado para gerar a resposta, no formato: O c√≥digo utilizado foi {pandas_instructions}"
)

print("‚úÖ Prompts configurados com sucesso!")

‚úÖ Prompts configurados com sucesso!


##4 Configura√ß√£o Pydantic V2

In [5]:
class LLMConfig(BaseModel):
    model: str = Field(..., description="Nome do modelo Groq a ser usado")
    api_key: str = Field(..., description="Chave da API Groq")
    data_url: str = Field(..., description="URL do CSV com os dados")

    @field_validator("data_url")
    @classmethod
    def validar_url(cls, v: str) -> str:
        if not (v.startswith("http://") or v.startswith("https://")):
            raise ValueError("data_url deve come√ßar com http:// ou https://")
        return v

    @field_validator("api_key")
    @classmethod
    def validar_api_key(cls, v: str) -> str:
        if not v or len(v.strip()) == 0:
            raise ValueError("api_key n√£o pode ser vazia")
        return v

print("‚úÖ Configura√ß√£o LLM com Pydantic criada!")

class Historico(BaseModel):
    entradas: List[Tuple[str, str]] = []

    def adicionar(self, pergunta: str, resposta: str):
        """Adiciona uma entrada ao hist√≥rico se pergunta e resposta forem v√°lidas."""
        if pergunta and resposta:
            self.entradas.append((pergunta, resposta))

print("‚úÖ Modelo de hist√≥rico com Pydantic criado!")

‚úÖ Configura√ß√£o LLM com Pydantic criada!
‚úÖ Modelo de hist√≥rico com Pydantic criado!


##5 Modelos de Eventos

(Rastrear cada passo da intera√ß√£o, como um hist√≥rico organizado)

In [6]:
# --- Modelos de Eventos com Pydantic ---

# --- Evento de in√≠cio customizado para garantir os dados de entrada ---
class StartEvent(BaseStartEvent):
    query: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

# --- Eventos intermedi√°rios para propagar os dados no workflow ---
class CodeEvent(Event):
    pandas_prompt: str
    query: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

class OutputEvent(Event):
    pandas_code: str
    query: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

class ExecutedEvent(Event):
    pandas_code: str
    pandas_output: Any
    query: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

print("‚úÖ Modelos de eventos do workflow definidos com sucesso!")

‚úÖ Modelos de eventos do workflow definidos com sucesso!


##6 Fun√ß√µes Auxiliares

In [9]:
def carregar_arquivo(caminho_arquivo: str, df_estado: Optional[pd.DataFrame] = None) -> Tuple[str, Optional[pd.DataFrame]]:
    """
    Carrega um CSV do caminho fornecido. Se caminho_arquivo for None, mant√©m df_estado.
    Retorna mensagem de status e DataFrame atualizado.
    """
    if not caminho_arquivo:
        return "Nenhum arquivo enviado. Usando dados atuais (se houver).", df_estado
    try:
        df_novo = pd.read_csv(caminho_arquivo)
        return "Arquivo carregado com sucesso!", df_novo
    except Exception as e:
        return f"Erro ao carregar arquivo: {str(e)}", df_estado

def formatar_texto(response: str, largura: int = 100, imprimir: bool = True):
    texto_formatado = textwrap.fill(response, width=largura)
    if imprimir:
        print(texto_formatado)
    return texto_formatado

def descricao_colunas(df: pd.DataFrame) -> str:
    descricao = "\n".join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return "Colunas do DataFrame:\n" + descricao

def limpar_codigo_pandas(codigo: str) -> str:
    """Limpa e valida o c√≥digo Pandas gerado pela LLM."""
    codigo = re.sub(r'```(?:python)?\n?', '', codigo) # Remove ```python
    codigo = re.sub(r'```', '', codigo) # Remove ``` de fechamento
    linhas = [linha.strip() for linha in codigo.split('\n') if linha.strip()]

    # Filtra linhas que s√£o coment√°rios ou texto explicativo
    codigo_filtrado = [
        linha for linha in linhas if not (
            linha.startswith('#') or
            'resposta:' in linha.lower() or
            'resultado:' in linha.lower()
        )
    ]

    if not codigo_filtrado:
        return ""

    # Se houver v√°rias linhas, a √∫ltima √© geralmente a express√£o de retorno
    return codigo_filtrado[-1]

print("‚úÖ Fun√ß√µes auxiliares configuradas!")

‚úÖ Fun√ß√µes auxiliares configuradas!


##7 Configura√ß√£o da API e Dados

In [17]:
# Configure sua chave da API Groq nos Secrets do Colab
# V√° em:  (√≠cone da chave) -> Add new secret
# Name: Groq_API
# Value: sua_chave_aqui

try:
    key = userdata.get("Groq_API")
    print("‚úÖ Chave API carregada com sucesso!")
except Exception as e:
    key = None
    print(f"‚ùå Erro ao carregar chave API: {e}. Por favor, configure a chave 'Groq_API' nos Secrets do Colab.")

# ===== CONFIGURA√á√ÉO E DADOS =====
if key:
    config = LLMConfig(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        api_key=key,
        data_url="https://raw.githubusercontent.com/YuriArduino/Estudos_Artificial_Intelligence/refs/heads/Dados/vendas.csv"
    )

    df = pd.read_csv(config.data_url)
    Settings.llm = Groq(model=config.model, api_key=config.api_key)

    print("\n Estrutura do DataFrame:")
    print(df.head())
    print(f"\n‚úÖ Configura√ß√£o completa! Dataset: {df.shape[0]} linhas, {df.shape[1]} colunas")
else:
    df = None
    print("\n‚ö†Ô∏è A execu√ß√£o ser√° interrompida pois a API Key n√£o foi carregada.")

‚úÖ Chave API carregada com sucesso!

 Estrutura do DataFrame:
     ID_compra filial       cidade tipo_cliente     genero       tipo_produto  \
0  750-67-8428      A  Santo Andr√©       Membro   Feminino     Sa√∫de e Beleza   
1  226-31-3081      C  S√£o Caetano       Normal   Feminino        Eletr√¥nicos   
2  631-41-3108      A  Santo Andr√©       Normal  Masculino               Casa   
3  123-19-1176      A  Santo Andr√©       Membro  Masculino     Sa√∫de e Beleza   
4  373-73-7910      A  Santo Andr√©       Normal  Masculino  Esportes e Viagem   

   preco_unitario  quantidade  imposto_5%     total        data      hora  \
0           74.69           7     26.1415  548.9715  2024-01-05  13:08:00   
1           15.28           5      3.8200   80.2200  2024-03-08  10:29:00   
2           46.33           7     16.2155  340.5255  2024-03-03  13:23:00   
3           58.22           8     23.2880  489.0480  2024-01-27  20:33:00   
4           86.31           7     30.2085  634.3785  2024

##8 Classe Workflow Principal

In [26]:
class PandasWorkflow(Workflow):

    @step
    async def iniciar_processamento(self, ev: StartEvent) -> CodeEvent:
        """
        Prepara o prompt para gerar o c√≥digo Pandas.
        Os dados 'query' e 'df' s√£o recebidos diretamente do StartEvent.
        """
        print("  [Workflow] Etapa 1: Iniciando processamento...")
        colunas_info = descricao_colunas(ev.df)
        prompt_text = pandas_prompt_str.format(
            colunas_detalhes=colunas_info,
            df_str=ev.df.head(5).to_string(),
            instruction_str=instruction_str,
            query_str=ev.query
        )
        return CodeEvent(pandas_prompt=prompt_text, query=ev.query, df=ev.df)

    @step
    async def gerar_codigo(self, ev: CodeEvent) -> OutputEvent:
        """Gera c√≥digo pandas a partir do prompt."""
        print("  [Workflow] Etapa 2: Gerando c√≥digo Pandas...")
        response = await Settings.llm.acomplete(ev.pandas_prompt)
        codigo_limpo = limpar_codigo_pandas(str(response).strip())
        print(f"   ‚úÖ C√≥digo gerado: {codigo_limpo}")
        return OutputEvent(pandas_code=codigo_limpo, query=ev.query, df=ev.df)

    @step
    async def executar_codigo(self, ev: OutputEvent) -> ExecutedEvent:
        """Executa o c√≥digo Pandas gerado."""
        print(" [Workflow] Etapa 3: Executando c√≥digo...")
        try:
            # Contexto seguro para a execu√ß√£o do eval
            contexto = {"df": ev.df, "pd": pd}
            resultado = eval(ev.pandas_code, {"__builtins__": {}}, contexto)
            print(f"   ‚úÖ Resultado da execu√ß√£o: {resultado}")
        except Exception as e:
            print(f"   ‚ùå Erro na execu√ß√£o: {e}")
            resultado = f"Erro ao executar o c√≥digo: {str(e)}"
        return ExecutedEvent(
            pandas_code=ev.pandas_code,
            pandas_output=resultado,
            query=ev.query,
            df=ev.df
        )

    @step
    async def finalizar_e_sintetizar(self, ev: ExecutedEvent) -> StopEvent:
        """Gera a resposta final em linguagem natural."""
        print("  [Workflow] Etapa 4: Sintetizando resposta final...")
        if isinstance(ev.pandas_output, str) and "Erro" in ev.pandas_output:
            resposta_final = f"N√£o foi poss√≠vel processar a consulta devido a um erro. {ev.pandas_output}"
        else:
            prompt_synthesis = RESPONSE_SYNTHESIS_PROMPT_STR.format(
                query_str=ev.query,
                pandas_instructions=ev.pandas_code,
                pandas_output=str(ev.pandas_output)
            )
            response = await Settings.llm.acomplete(prompt_synthesis)
            resposta_final = str(response).strip()

        print(" [Workflow] Finalizado com sucesso!")
        return StopEvent(result={
            "resposta_final": resposta_final,
            "pandas_code": ev.pandas_code,
        })

print("‚úÖ Workflow robusto e simplificado definido!")

‚úÖ Workflow robusto e simplificado definido!


##9 Fun√ß√£o de Execu√ß√£o

In [22]:
async def executar_consulta(query: str, df_local: pd.DataFrame):
    """
    Executa o workflow de ponta a ponta e trata os erros de forma expl√≠cita, sem fallback.
    """
    if df_local is None:
        return {
            "resposta_final": "Erro: O DataFrame n√£o foi carregado.",
            "pandas_code": "N/A"
        }
    if not query or not query.strip():
        return {
            "resposta_final": "Erro: A consulta n√£o pode ser vazia.",
            "pandas_code": "N/A"
        }

    try:
        print(f"\n Iniciando workflow para a consulta: '{query}'")
        wf = PandasWorkflow()

        # O m√©todo run() est√° retornando o dicion√°rio diretamente.
        # A corre√ß√£o √© tratar o resultado como o dicion√°rio final.
        resultado_final = await wf.run(query=query, df=df_local)

        # Verificamos se o resultado √© de fato um dicion√°rio, como esperado.
        if not isinstance(resultado_final, dict):
             raise TypeError(f"O workflow retornou um tipo inesperado: {type(resultado_final)}")

        return resultado_final

    except Exception as e:
        # SEM FALLBACK: Se o workflow falhar, reportamos o erro diretamente.
        print(f"‚ùå Erro cr√≠tico durante a execu√ß√£o do workflow: {e}")
        import traceback
        traceback.print_exc() # Imprime o stack trace completo para depura√ß√£o
        return {
            "resposta_final": f"Ocorreu um erro cr√≠tico no workflow. Verifique os logs para detalhes. Erro: {str(e)}",
            "pandas_code": "N/A - Falha no workflow"
        }


print("‚úÖ Fun√ß√£o de execu√ß√£o do workflow corrigida: sem fallback e com tratamento de erro expl√≠cito!")

‚úÖ Fun√ß√£o de execu√ß√£o do workflow corrigida: sem fallback e com tratamento de erro expl√≠cito!


##9.2 Fun√ß√µes auxiliares Gradio

In [15]:
def carregar_dados(file, df_estado):
    if file is None:
        return "‚ùå Nenhum arquivo selecionado", df_estado
    try:
        df_local = pd.read_csv(file.name) # Corrigido para .name
        return f"‚úÖ Arquivo carregado! {df_local.shape[0]} linhas, {df_local.shape[1]} colunas.", df_local
    except Exception as e:
        return f"‚ùå Erro ao carregar arquivo: {str(e)}", df_estado

def processar_pergunta(pergunta: str, df_local: Optional[pd.DataFrame], historico: Historico):
    if df_local is None:
        resposta = "‚ùå Por favor, carregue um arquivo CSV primeiro."
        return resposta, historico

    if not pergunta or not pergunta.strip():
        resposta = "‚ùå Por favor, digite uma pergunta v√°lida."
        return resposta, historico

    try:
        # Executa a fun√ß√£o ass√≠ncrona principal
        resultado = asyncio.run(executar_consulta(pergunta, df_local))
        resposta = resultado.get("resposta_final", "‚ùå Erro: a resposta n√£o foi gerada.")
        historico.adicionar(pergunta, resposta)
        return resposta, historico
    except Exception as e:
        erro_msg = f"‚ùå Ocorreu um erro inesperado: {str(e)}"
        historico.adicionar(pergunta, erro_msg)
        return erro_msg, historico

def gerar_pdf(historico: Historico):
    if not historico.entradas:
        return "‚ùå Nenhum hist√≥rico para gerar o PDF."

    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        caminho_pdf = f"relatorio_consultas_{timestamp}.pdf"

        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=10) # Usar UTF-8 para caracteres especiais
        pdf.add_font('Arial', '', 'Arial.ttf', uni=True)


        # T√≠tulo
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Relat√≥rio de Consultas ao CSV", ln=True, align="C")
        pdf.ln(10)

        for i, (pergunta, resposta) in enumerate(historico.entradas, 1):
            # Pergunta
            pdf.set_font("Arial", "B", 12)
            pdf.multi_cell(0, 8, f"Pergunta {i}: {pergunta}".encode('latin-1', 'replace').decode('latin-1'))
            pdf.ln(2)

            # Resposta
            pdf.set_font("Arial", "", 10)
            pdf.multi_cell(0, 6, f"Resposta: {resposta}".encode('latin-1', 'replace').decode('latin-1'))
            pdf.ln(8)

        pdf.output(caminho_pdf)
        return f"‚úÖ PDF gerado com sucesso: {caminho_pdf}"
    except Exception as e:
        return f"‚ùå Erro ao gerar PDF: {str(e)}"

print("‚úÖ Fun√ß√µes auxiliares do Gradio prontas!")

‚úÖ Fun√ß√µes auxiliares do Gradio prontas!


##9.1 Gradio

In [20]:
import gradio as gr

with gr.Blocks(title="Pandas CSV Analyzer", theme=gr.themes.Soft()) as app:
    gr.Markdown("# üêº Analisador de CSV com Pandas e LlamaIndex")
    gr.Markdown("Fa√ßa upload de um arquivo CSV e fa√ßa perguntas em linguagem natural!")

    # Estados (invis√≠veis) para armazenar o DataFrame e o hist√≥rico
    df_estado = gr.State()
    historico_estado = gr.State(value=Historico())

    with gr.Row():
        with gr.Column(scale=1):
            input_arquivo = gr.File(label="üìÅ Fa√ßa upload do seu arquivo CSV", file_types=[".csv"])
            upload_status = gr.Textbox(label="üìä Status do Upload", interactive=False)
            botao_pdf = gr.Button("üìÑ Gerar Relat√≥rio PDF", variant="secondary")
            output_pdf = gr.Textbox(label="üìÅ Status do PDF", interactive=False)

        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="Hist√≥rico da Conversa", height=500)
            input_pergunta = gr.Textbox(label="‚ùì Digite sua pergunta", placeholder="Ex: Qual o total de vendas por filial?")
            botao_enviar = gr.Button(" Enviar", variant="primary")

    def chat_interface(pergunta, chat_history, df_local, historico_obj):
        if df_local is None:
            chat_history.append((pergunta, "‚ùå Por favor, carregue um arquivo CSV primeiro."))
            return "", chat_history, historico_obj

        # Chama a fun√ß√£o de processamento principal
        resposta, historico_obj_atualizado = processar_pergunta(pergunta, df_local, historico_obj)
        chat_history.append((pergunta, resposta))
        return "", chat_history, historico_obj_atualizado

    # --- Conex√µes dos Eventos ---
    input_arquivo.upload(
        fn=carregar_dados,
        inputs=[input_arquivo, df_estado],
        outputs=[upload_status, df_estado]
    )

    botao_enviar.click(
        fn=chat_interface,
        inputs=[input_pergunta, chatbot, df_estado, historico_estado],
        outputs=[input_pergunta, chatbot, historico_estado]
    )

    input_pergunta.submit(
        fn=chat_interface,
        inputs=[input_pergunta, chatbot, df_estado, historico_estado],
        outputs=[input_pergunta, chatbot, historico_estado]
    )

    botao_pdf.click(
        fn=gerar_pdf,
        inputs=[historico_estado],
        outputs=[output_pdf]
    )

print("‚úÖ Interface Gradio configurada!")

# Lan√ßa a interface
if __name__ == "__main__" and df is not None:
    app.launch(debug=True, share=False)
elif df is None:
    print("‚ùå A interface Gradio n√£o ser√° iniciada pois a configura√ß√£o da API falhou.")

/tmp/ipython-input-2411074614.py:19: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Hist√≥rico da Conversa", height=500)


‚úÖ Interface Gradio configurada!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>


üöÄ Iniciando workflow para a consulta: 'Qual √© a quantidade de registros no arquivo?'
‚û°Ô∏è  [Workflow] Etapa 1: Iniciando processamento...
‚û°Ô∏è  [Workflow] Etapa 2: Gerando c√≥digo Pandas...
   ‚úÖ C√≥digo gerado: df.shape[0]
‚û°Ô∏è  [Workflow] Etapa 3: Executando c√≥digo...
   ‚úÖ Resultado da execu√ß√£o: 1000
‚û°Ô∏è  [Workflow] Etapa 4: Sintetizando resposta final...
üèÅ [Workflow] Finalizado com sucesso!
Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 

##10 Teste R√°pido

In [27]:
# Teste r√°pido para verificar se a fun√ß√£o principal est√° funcionando
async def teste_rapido():
    if df is not None:
        query_teste = "Qual √© a avalia√ß√£o m√©dia de cada filial?"
        # Corrigido: passando o df_local como argumento
        resultado = await executar_consulta(query_teste, df_local=df)

        if resultado and "Erro" not in resultado.get("resposta_final", ""):
            print("\n‚úÖ Sistema funcionando perfeitamente!")
            print(f"Resposta: {resultado['resposta_final']}")
        else:
            print(f"\n‚ùå Houve um problema no teste: {resultado}")
    else:
        print("‚ö†Ô∏è Teste n√£o executado: DataFrame n√£o foi carregado.")

# Executar o teste
await teste_rapido()


 Iniciando workflow para a consulta: 'Qual √© a avalia√ß√£o m√©dia de cada filial?'
  [Workflow] Etapa 1: Iniciando processamento...
  [Workflow] Etapa 2: Gerando c√≥digo Pandas...
   ‚úÖ C√≥digo gerado: df.groupby('filial')['avaliacao'].mean()
 [Workflow] Etapa 3: Executando c√≥digo...
   ‚úÖ Resultado da execu√ß√£o: filial
A    7.027059
B    6.818072
C    7.072866
Name: avaliacao, dtype: float64
  [Workflow] Etapa 4: Sintetizando resposta final...
 [Workflow] Finalizado com sucesso!

‚úÖ Sistema funcionando perfeitamente!
Resposta: A avalia√ß√£o m√©dia de cada filial √© a seguinte: 

- Filial A: 7,027059
- Filial B: 6,818072
- Filial C: 7,072866

Esses valores indicam que a Filial C tem a melhor avalia√ß√£o m√©dia, seguida de perto pela Filial A. A Filial B tem a avalia√ß√£o m√©dia mais baixa entre as tr√™s.

O c√≥digo utilizado foi df.groupby('filial')['avaliacao'].mean()


##11 Exemplos de Uso Individual

In [ ]:
# Execute uma consulta por vez para testar

# Exemplo 1
query = "Quais s√£o os 5 produtos mais vendidos?"
result = await executar_consulta(query)

In [ ]:
# Exemplo 2
query = "Qual o total de vendas por regi√£o?"
result = await executar_consulta(query)

In [ ]:
# Exemplo 3
query = "Qual produto tem a maior margem de lucro?"
result = await executar_consulta(query)

In [ ]:
# Exemplo 4
query = "Quantas vendas foram feitas em cada m√™s?"
result = await executar_consulta(query)

In [ ]:
query = "Qual o gen√™ro que mais consome e o segmento do produto?"
result = await executar_consulta(query)

##12 Bateria de Testes

In [ ]:
# Executar m√∫ltiplas consultas de uma vez
consultas_exemplo = [
    "Qual √© a avalia√ß√£o m√©dia de cada filial?",
    "Quais s√£o os 5 produtos mais vendidos?",
    "Qual o total de vendas por regi√£o?",
    "Qual produto tem a maior margem de lucro?",
    "Quantas vendas foram feitas em cada m√™s?",
    "Qual √© a receita total por categoria de produto?",
    "Qual filial teve melhor desempenho?",
    "Mostre a distribui√ß√£o de pre√ßos dos produtos"
]

print(" Executando bateria de testes...")
resultados = await executar_multiplas_consultas(consultas_exemplo)

print(f"\n‚úÖ Conclu√≠do! {len([r for r in resultados if r])} consultas processadas com sucesso.")

##13 Modo Interativo

In [ ]:
# Modo interativo - digite suas pr√≥prias consultas
print(" MODO INTERATIVO")
print("Digite suas consultas (digite 'sair' para terminar)")
print("-" * 50)

while True:
    try:
        query = input("\n Sua consulta: ").strip()

        if query.lower() in ['sair', 'exit', 'quit', '']:
            print(" At√© logo!")
            break

        await executar_consulta(query, mostrar_detalhes=False)

    except KeyboardInterrupt:
        print("\n Interrompido pelo usu√°rio!")
        break
    except Exception as e:
        print(f" Erro: {e}")

##14 An√°lise Manual dos Dados


In [ ]:
# An√°lise dos dados originais para compara√ß√£o
print(" AN√ÅLISE MANUAL DOS DADOS (para compara√ß√£o)")
print("="*50)

print(f"Estat√≠sticas b√°sicas:")
print(df.describe())

print(f"\nContagem por categoria:")
for col in df.select_dtypes(include=['object']).columns:
    print(f"\n{col}:")
    print(df[col].value_counts().head())

print(f"\nValores nulos:")
print(df.isnull().sum())

#Dicas e Informa√ß√µes

In [ ]:
print("üí° DICAS PARA USO:")
print("="*50)
print("1. Para consultas complexas, seja espec√≠fico")
print("2. Use nomes exatos das colunas mostradas na an√°lise")
print("3. Experimente diferentes tipos de an√°lise:")
print("   - Agrupamentos: 'm√©dia por categoria'")
print("   - Rankings: 'top 10 produtos'")
print("   - Filtros: 'vendas acima de X valor'")
print("   - S√©ries temporais: 'vendas por m√™s'")
print("4. O sistema mostra o c√≥digo Pandas usado!")

print(f"\n INFORMA√á√ïES DO DATASET:")
print(f"Suas colunas: {list(df.columns)}")
print(f"Quantidade de dados: {df.shape[0]} linhas x {df.shape[1]} colunas")
print(f"Tipos de dados: {df.dtypes.value_counts().to_dict()}")

print(f"\n SUGEST√ïES DE CONSULTAS:")
example_queries = [
    "Qual a correla√ß√£o entre pre√ßo e quantidade vendida?",
    "Mostre a distribui√ß√£o de vendas por dia da semana",
    "Qual filial tem o melhor desempenho?",
    "Compare as vendas do primeiro e √∫ltimo trimestre",
    "Identifique produtos com baixo estoque",
    "Qual √© o produto mais caro por categoria?",
    "Mostre as vendas m√©dias por vendedor",
    "Qual regi√£o tem menor varia√ß√£o de pre√ßos?"
]

for i, query in enumerate(example_queries, 1):
    print(f"{i}. {query}")

print(f"\n MODELO UTILIZADO: {config.model}")
print(f" URL DOS DADOS: {config.data_url}")

#Notas de Uso:

*   Configura√ß√£o da API: Configure sua chave Groq nos Secrets do Colab
*   Execu√ß√£o sequencial: Execute as c√©lulas na ordem
*   Personaliza√ß√£o: Modifique a URL dos dados na C√©lula 6
*   Debug: Os logs mostram o c√≥digo Pandas gerado
*   Interativo: Use a C√©lula 12 para consultas livres

##Estrutura final:

*   Instala√ß√£o autom√°tica de depend√™ncias
* Configura√ß√£o completa da API
* Explora√ß√£o autom√°tica dos dados
* Sistema de workflow robusto
* Exemplos prontos para uso
* Modo interativo
* Sistema de debug integrado


>Desenvolvido por Yuri Arduino Bernardineli Alves | GitHub: [YuriArduino](https://github.com/YuriArduino) | E-mail: yuriarduino@gmail.com